In [55]:
from dotenv import load_dotenv
import os
# import openai
# import anthropic
# import google.generativeai
import requests
from IPython.display import Markdown, display,update_display

In [56]:
# Load environment variables from .env
load_dotenv()

# Define your API keys and expected prefixes
api_keys = {
    "OPEN_AI_API_KEY": "sk-proj-",
    "ANTHROPIC_API_KEY": "sk-ant-",
    "GOOGLE_GEMINI_API_KEY": "gems-"
}

def validate_key(name, prefix):
    key = os.getenv(name)
    if key and key.startswith(prefix):
        print(f"✅ {name} is valid.")
    elif key:
        print(f"⚠️ {name} is present but may be malformed.")
    else:
        print(f"❌ {name} is missing from environment.")
    return key

# Validate and assign keys
openai_key = validate_key("OPEN_AI_API_KEY", api_keys["OPEN_AI_API_KEY"])
anthropic_key = validate_key("ANTHROPIC_API_KEY", api_keys["ANTHROPIC_API_KEY"])
gemini_key = validate_key("GOOGLE_GEMINI_API_KEY", api_keys["GOOGLE_GEMINI_API_KEY"])

# Assign keys to libraries or environment if needed
if openai_key:
    import openai
    openai.api_key = openai_key

if anthropic_key:
    import anthropic
    client = anthropic.Client(api_key=anthropic_key)

if gemini_key:
    import google.generativeai as genai
    genai.configure(api_key=gemini_key)

    os.environ["GOOGLE_GEMINI_API_KEY"] = gemini_key

✅ OPEN_AI_API_KEY is valid.
✅ ANTHROPIC_API_KEY is valid.
⚠️ GOOGLE_GEMINI_API_KEY is present but may be malformed.


ModuleNotFoundError: No module named 'anthropic'

In [57]:
system_prompt = "You are an assistant that is great at telling jokes"
user_prompt = "Tell a light-hearted joke for an audience of Data Scientists"

In [58]:
prompts = [
    {"role":"system","content":system_prompt},
    {"role":"use","content":user_prompt},
    
]

In [61]:
#GPT-3.5-Turbo
completions = openai.ChatCompletion.create(model="gpt-3.5-turbo", messages=prompts)
print(completions.choices[0].message.content)

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:
#GPT-4O-MINI
# Temperature setting controls creativity
completions= openai.ChatCompletions.create(model="gpt-4o-mini",messages=prompts,temperature=0.7)
print(completions.choices[0].message.content)

AttributeError: module 'openai' has no attribute 'Chat'

In [29]:
#GPT-4O
completions= openai.chat.completions.create(model="gpt-4o",messages=prompts,temperature = 0.4)
print(completions.choices[0].message.content)

AuthenticationError: Error code: 401 - {'error': {'message': "Incorrect API key provided: <module *******************************************************************************************************************py'>. You can find your API key at https://platform.openai.com/account/api-keys.", 'type': 'invalid_request_error', 'param': None, 'code': 'invalid_api_key'}}

In [48]:
# Cloude 3.5 Sonnet
#Api needs systems message provided separately from user prompt 
# Also adding max_token

message = client.messages.create(
    model="cloude-3.5-sonnet-2024620",
    max_token = 200,
    temperature=0.7,
    system=system_message,
    messages=[
        {"role":"user","content":user_prompt}
    ]
)
print(message.content[0].text)

AttributeError: 'str' object has no attribute 'messages'

In [ ]:
# Cloude 3.5 Sonnet again
# Now let's add in streaming back results
result = cloude.messages.stream(model= "cloude-3.5-sonnet-20240620",max_token=200, temperature=0.7, system=system_message,messages=[{"role":"user", "content":user_prompt}])

with result as stream:
    for text in stream.text_stream:
        print(text,end="",flush=True)

In [ ]:
# The API for Gemini has a slightly different structure
gemini = google.generativeai.GenerativeModel(model_name="gemini-1.5-flush",system_instruction = system_message)
response = gemini.generate_content(user_prompt)
print(response.text)

In [ ]:
prompts = [
    {"role":"system", "content":"You are a helpful assistant"},
    {"role":"user","content":"How do i decide if a business problem is suitable for an LLM solution"}
]

In [63]:
# Have is stream back result in markdown
stream = openai.chat.completions.create(model="gpt-4o",messages=prompts,temperature=0.7,stream=True)
reply=""
display_handle = display(Markdown(""),display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or""
    reply= reply.replace("```", "").replace("markdown","")
    update_display(Markdown(reply), display_id=display_handle.display_id)

BadRequestError: Error code: 400 - {'error': {'message': "Invalid value: 'use'. Supported values are: 'system', 'assistant', 'user', 'function', 'tool', and 'developer'.", 'type': 'invalid_request_error', 'param': 'messages[1].role', 'code': 'invalid_value'}}